# **ResNet**
此份程式碼會介紹如何使用 tf.keras 的方式建構 ResNet 的模型架構。

![image](https://hackmd.io/_uploads/rJ8SA1HOp.png)

- [source paper](https://arxiv.org/abs/1512.03385)

## 匯入套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow.keras import datasets, layers, Model, Sequential, losses

## 載入資料集

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Expand dimensions
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Grayscale to RGB
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Split dataset into training and validation data
x_val = x_train[int(x_train.shape[0]*0.8):, :, :, :]
y_val = y_train[int(y_train.shape[0]*0.8):]
x_train = x_train[:int(x_train.shape[0]*0.8), :, :, :]
y_train = y_train[:int(y_train.shape[0]*0.8)]
print(f'x_train shape: {x_train.shape}, x_val shape: {x_val.shape}')
print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}')

## ResNet Arhietecture

![image](https://hackmd.io/_uploads/B16H0kHOT.png)

- [source paper](https://arxiv.org/abs/1512.03385)

In [ ]:
def ResBlock(inputs, blocks_num, filters_num, kernel_size, strides=1):
    for i in range(blocks_num):
        x = layers.Conv2D(filters_num[0],
                          (kernel_size[0], kernel_size[0]),
                          strides=strides,
                          padding='same')(inputs)
        x = layers.BatchNormalization()(x)
        strides = 1
        for j in range(1, len(filters_num)):
            x = layers.Activation('relu')(x)
            x = layers.Conv2D(filters_num[j],
                              (kernel_size[j], kernel_size[j]),
                              strides=strides,
                              padding='same')(x)
            x = layers.BatchNormalization()(x)

        # 確認 Skip connection 維度一致
        if inputs.shape.as_list() == x.shape.as_list():
            identity = inputs
        else:
            identity_strides = inputs.shape[1]//x.shape[1]
            identity = layers.Conv2D(filters_num[-1], (1, 1),
                                     strides=identity_strides)(inputs)
            identity = layers.BatchNormalization()(identity)

        outputs = layers.add([identity, x])
        outputs = layers.Activation('relu')(outputs)
        inputs = outputs

    return outputs

In [ ]:
labels_num = 10

In [ ]:
tf.keras.backend.clear_session()
inputs = layers.Input(shape=x_train.shape[1:])
x = layers.Resizing(224, 224,
                    interpolation="bilinear",
                    input_shape=x_train.shape[1:])(inputs)
x = layers.Conv2D(64, (7, 7), strides=2, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D((3, 3), strides=2, padding='same')(x)

# conv2_x
x = ResBlock(x, 3, [64, 64, 256], [1, 3, 1], strides=1)
# conv3_x
x = ResBlock(x, 4, [128, 128, 512], [1, 3, 1], strides=2)
# conv4_x
x = ResBlock(x, 6, [256, 256, 1024], [1, 3, 1], strides=2)
# conv5_x
x = ResBlock(x, 3, [512, 512, 2048], [1, 3, 1], strides=2)

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(labels_num)(x)

In [ ]:
ResNet_model = Model(inputs = inputs, outputs = outputs)

In [ ]:
ResNet_model.summary()

In [ ]:
batch_size = 256

In [ ]:
inputs = np.ones((batch_size, x_train.shape[1], x_train.shape[2], 3),
                 dtype=np.float32)
ResNet_model(inputs)